In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from lifelines import KaplanMeierFitter, CoxPHFitter
from lifelines.statistics import multivariate_logrank_test
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests

In [76]:
# Read in metadata and feature matrix
# Change to your own paths; feature_mat should have patient_id as rows and features as columns
metadata = pd.read_csv("~/Desktop/BNFO285_Projects/Data/processed/processed_metadata.txt", sep="\t", header=0)
metadata = metadata[metadata["hpv_status"] != "positive"]
metadata = metadata[metadata["tumor_site"] != "tonsil"]
res_dir = "/Users/clairez/Desktop/BNFO285_Projects/project_4/Wenshu/test/test_results/"
feature_tab = pd.read_csv("/Users/clairez/Desktop/BNFO285_Projects/Data/processed/filtered_cnv_status.tsv",
                          sep="\t", header=0, index_col=0)
feature_mat = feature_tab.T.copy()
feature_mat.index.name = "patient_id"
feature_mat

gene_name,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
patient_id,,,,,,,,,,,,,,,,,,,,,
TCGA-CN-A642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-UF-A7JH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-UF-A7JO,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0
TCGA-CV-7180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0
TCGA-CV-6942,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-BB-A6UO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-CR-6480,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
TCGA-DQ-5629,0.0,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,-1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0


In [73]:
# Filtering step for CNV matrix
count_tab = feature_mat.apply(lambda col: col.value_counts())
count_tab.index.name = "status"
count_tab

gene_name,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
status,,,,,,,,,,,,,,,,,,,,,
-1.0,69,80,38,38,67,65,17,45,53,18,...,74,225,93,96,26,63,63,109,111,85
0.0,363,348,347,346,390,378,223,410,409,188,...,390,236,371,372,259,399,399,341,349,384
1.0,56,12,104,105,32,46,249,34,27,283,...,25,28,25,21,204,27,27,39,29,19


In [77]:
feature_mat[feature_mat == -1] = 1
freq = feature_mat.mean(axis=0).sort_values(ascending=True)
freq
keep = freq[(freq >= 0.05) & (freq <= 0.95)].index
feature_mat = feature_mat[keep]
feature_mat

gene_name,CYP26B1,NAGK,MCEE,PAIP2B,ZNF638,MPHOSPH10,FIGLA,TEX261,ADD2,DYSF,...,CACNA2D2,RBM5,GNAT1,DOCK3,CISH,GRM2,HEMK1,GNAI2,IQCF3,SEMA3F
patient_id,,,,,,,,,,,,,,,,,,,,,
TCGA-CN-A642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-UF-A7JH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
TCGA-UF-A7JO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
TCGA-CV-7180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
TCGA-CV-6942,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-BB-A6UO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
TCGA-CR-6480,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
TCGA-DQ-5629,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [78]:
features = feature_mat.columns.tolist()

df = pd.merge(metadata, feature_mat, on="patient_id")
df = pd.get_dummies(df, columns=['gender'], drop_first=True)
df

,patient_id,age,OS,OS_time,DSS,DSS_time,DFI,DFI_time,tumor_site,tumor_class,...,RBM5,GNAT1,DOCK3,CISH,GRM2,HEMK1,GNAI2,IQCF3,SEMA3F,gender_MALE
0,TCGA-4P-AA8J,66.0,0.0,102.0,0.0,102.0,NaN,NaN,tongue,primary,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,True
1,TCGA-BA-4074,69.0,1.0,462.0,1.0,462.0,NaN,NaN,tongue,primary,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,True
2,TCGA-BA-4075,49.0,1.0,283.0,1.0,283.0,NaN,NaN,tongue,primary,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,True
3,TCGA-BA-4076,39.0,1.0,415.0,1.0,415.0,NaN,NaN,Larynx,primary,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,True
4,TCGA-BA-4077,45.0,1.0,1134.0,1.0,1134.0,NaN,NaN,tongue,primary,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,TCGA-UF-A7JT,72.0,1.0,993.0,1.0,993.0,NaN,NaN,mouth,metastasis,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
442,TCGA-UF-A7JV,62.0,1.0,90.0,1.0,90.0,NaN,NaN,Hypopharynx,recurrence,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
443,TCGA-UP-A6WW,58.0,0.0,518.0,0.0,518.0,NaN,NaN,tongue,primary,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
444,TCGA-WA-A7GZ,58.0,1.0,625.0,0.0,625.0,0.0,625.0,mouth,primary,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,True


In [90]:
# Univariate Cox Proportional Hazards Models
def univariate_cox(df, data_type, q_thresh = 0.05):
    results = []
    for feature in features:
        if(data_type in ("cna", "mutation")):
            feat = df[feature].dropna().astype(str)
            # one-hot encode this feature only
            #if(data_type == "cna"):
            #   feat = pd.Categorical(feat, categories=["neutral", "gain", "loss"], ordered=True)
            dummies = pd.get_dummies(feat, prefix=feature, drop_first=True)
            cols = ['OS_time','OS'] + list(dummies.columns)

            data = pd.concat([df[['OS_time','OS']], dummies], axis=1).loc[:, cols].dropna()
            
            cph = CoxPHFitter()
            cph.fit(data,
                    duration_col='OS_time',
                    event_col='OS',
                    show_progress=False)
            
            for cov in dummies.columns:
                summ = cph.summary.loc[cov]
                results.append({
                    'feature':   feature,
                    'covariate': cov,
                    'coef':      summ['coef'],
                    'exp(coef)': summ['exp(coef)'],
                    'zscore':    summ['z'],
                    'p':     summ['p'],
                    'lower95%':  summ['exp(coef) lower 95%'],
                    'upper95%':  summ['exp(coef) upper 95%']
                })
        else:
            data = df[[feature, 'OS_time', 'OS']].dropna()

            cph = CoxPHFitter()
            cph.fit(data, duration_col='OS_time', event_col='OS', show_progress=False)
            summ = cph.summary.loc[feature]

            results.append({
                'feature':     feature,
                'coef':        summ['coef'],
                'exp(coef)':   summ['exp(coef)'],
                'zscore':      summ['z'],
                'p':           summ['p'],
                'lower 95%':   summ['exp(coef) lower 95%'],
                'upper 95%':   summ['exp(coef) upper 95%']
            })

    res_df = pd.DataFrame(results).set_index('feature')
    res_df["q"] = multipletests(res_df['p'], method='fdr_bh')[1]
    res_df = res_df.sort_values('q', ascending=True)
    res_df.to_csv(f'{res_dir}{data_type}_univariate_cox_results.tsv', sep="\t", index=True, header=True)
    sig_features = res_df[res_df['q'] < q_thresh]
    sig_features.to_csv(f'{res_dir}{data_type}_univariate_cox_significant_features.tsv', sep="\t", index=True, header=True)
    return sig_features


In [95]:
# Multivariate Cox Proportional Hazards Models
def multivariate_cox(df, data_type, q_thresh = 0.05):
    results = []
    for feature in features:
        if(data_type in ("cna", "mutation")):
            feat = df[feature].dropna().astype(str)
            # one-hot encode this feature only
            dummies = pd.get_dummies(feat, prefix=feature, drop_first=True)
            cols = ['OS_time','OS','age', 'tumor_stage'] \
                + [c for c in df if c.startswith('gender_')] \
                + list(dummies.columns)

            data = pd.concat([df[['OS_time','OS','age', 'tumor_stage'] + 
                             [c for c in df if c.startswith('gender_')]],
                            dummies], axis=1).loc[:, cols].dropna()
            
            cph = CoxPHFitter()
            cph.fit(data, duration_col='OS_time', event_col='OS', show_progress=False)
            
            for cov in dummies.columns:
                summ = cph.summary.loc[cov]
                results.append({
                    'feature':   feature,
                    'covariate': cov,
                    'coef':      summ['coef'],
                    'exp(coef)': summ['exp(coef)'],
                    'zscore':    summ['z'],
                    'p':     summ['p'],
                    'lower95%':  summ['exp(coef) lower 95%'],
                    'upper95%':  summ['exp(coef) upper 95%']
                })
        else:
            cols = ['OS_time','OS','age', 'tumor_stage'] \
                + [c for c in df.columns if c.startswith('gender_')] \
                + [feature]
            data = df[cols].dropna()

            cph = CoxPHFitter()
            cph.fit(data, duration_col='OS_time', event_col='OS', show_progress=False)
            summ = cph.summary.loc[feature]

            results.append({
                'feature':     feature,
                'coef':        summ['coef'],
                'exp(coef)':   summ['exp(coef)'],
                'zscore':      summ['z'],
                'p':           summ['p'],
                'lower 95%':   summ['exp(coef) lower 95%'],
                'upper 95%':   summ['exp(coef) upper 95%']
            })

    res_df = pd.DataFrame(results).set_index('feature')
    res_df["q"] = multipletests(res_df['p'], method='fdr_bh')[1]
    res_df = res_df.sort_values('q', ascending=True)
    res_df.to_csv(f'{res_dir}{data_type}_multivariate_cox_results.tsv', sep="\t", index=True, header=True)
    sig_features = res_df[res_df['q'] < q_thresh]
    sig_features.to_csv(f'{res_dir}{data_type}_multivariate_cox_significant_features.tsv', sep="\t", index=True, header=True)
    return sig_features

In [82]:
univariate_results = univariate_cox(df, "cna", q_thresh = 0.05)

In [96]:
multiivariate_results = multivariate_cox(df, "cna", q_thresh = 0.05)